In [20]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [21]:
df = pd.read_csv('data.csv')
df


#Incidência Pélvica - Medida angular da inclinação da pelve em relação à vertical
#Inclinação Pélvica Numérica - representa a inclinação da pelve, mas de uma maneira numérica.
#Ângulo de Lordose Lombar - Ângulo formado pela curvatura natural da parte inferior da coluna vertebral, conhecida como lordose lombar. 
#Ângulo do Declive Sacral - Ângulo formado pela inclinação do sacro, osso triangular na base da coluna vertebral, em relação à horizontal.
#Raio Pélvico - Medida do raio da pelve, geralmente relacionada à largura da pelve.
#Grau de Espondilolistese - Medida do deslizamento de uma vértebra sobre a outra.
#Classe - Variável de saída ou rótulo que indica a classe à qual pertence uma observação.

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
0,63.027817,22.552586,39.609117,40.475232,98.672917,-0.254400,Abnormal
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,Abnormal
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,Abnormal
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,Abnormal
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,Abnormal
...,...,...,...,...,...,...,...
305,47.903565,13.616688,36.000000,34.286877,117.449062,-4.245395,Normal
306,53.936748,20.721496,29.220534,33.215251,114.365845,-0.421010,Normal
307,61.446597,22.694968,46.170347,38.751628,125.670725,-2.707880,Normal
308,45.252792,8.693157,41.583126,36.559635,118.545842,0.214750,Normal


In [22]:
# Tratamento de elementos faltantes (assumindo que não há dados faltantes)
print(f"Elementos faltantes:\n{df.isna().sum()}\n")

# Convertendo a variável 'class' em numérica
df['class'] = df['class'].map({'Abnormal': 1, 'Normal': 0})

# Normalização das variáveis independentes
colunas_independentes = ["pelvic_incidence", "pelvic_tilt numeric", "lumbar_lordosis_angle", "sacral_slope", "pelvic_radius", "degree_spondylolisthesis"]
scaler = MinMaxScaler()
df[colunas_independentes] = scaler.fit_transform(df[colunas_independentes])

# Balanceamento das classes
classe_maioria = df[df["class"] == 0] #contém apenas as amostras da classe majoritária, onde o rótulo da classe é igual a 0.
classe_minoria = df[df["class"] == 1] #contém apenas as amostras da classe minoritária, onde o rótulo da classe é igual a 1.
classe_maioria = resample(classe_maioria, n_samples=len(classe_minoria)) # realizar oversampling (aumentar o número de amostras) na classe majoritária para equilibrar com a minoritária
df = pd.concat([classe_maioria, classe_minoria]) #contém o conjunto de dados original com as classes balanceadas.
df = df.sample(frac=1)  #Embaralha todas as amostras do DataFrame e, com isso, todas as linhas são incluídas.

# Separação dos Dados  30%teste e 70%treinamento
X_train, X_test, y_train, y_test = train_test_split(df[colunas_independentes], df["class"], test_size=0.3, random_state=42) #fixada em 42 para garantir reprodutibilidade.

Elementos faltantes:
pelvic_incidence            0
pelvic_tilt numeric         0
lumbar_lordosis_angle       0
sacral_slope                0
pelvic_radius               0
degree_spondylolisthesis    0
class                       0
dtype: int64



In [23]:
#contém o conjunto de dados original com as classes balanceadas.
#Embaralha todas as amostras do DataFrame e, com isso, todas as linhas são incluídas.

df

,pelvic_incidence,pelvic_tilt numeric,lumbar_lordosis_angle,sacral_slope,pelvic_radius,degree_spondylolisthesis,class
255,0.399188,0.378839,0.393776,0.365679,0.575874,0.086192,0
283,0.220358,0.351311,0.338936,0.208353,0.605619,0.026987,0
229,0.166739,0.297404,0.197170,0.184835,0.724360,0.018491,0
147,0.279043,0.049923,0.375865,0.420809,0.428363,0.099701,1
4,0.227272,0.289479,0.128129,0.247022,0.409579,0.044173,1
...,...,...,...,...,...,...,...
242,0.124793,0.215971,0.115731,0.186778,0.661392,0.023973,0
199,0.474032,0.414855,0.423261,0.418832,0.524805,0.099241,1
262,0.157874,0.373849,0.101336,0.136723,0.632585,0.027503,0
122,0.520098,0.975667,0.343678,0.172478,0.436907,0.183392,1


In [24]:
#Perceptron
# Modelo Perceptron em TensorFlow/Keras
modelo_p = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Camada de entrada com o número de características
    tf.keras.layers.Dense(1, activation='sigmoid')  # Camada de saída com 1 neurônio (classificação binária)
])

# Função de perda para problemas de classificação binária.
# Métrica para monitorar durante o treinamento, neste caso, a acurácia.
modelo_p.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Número de épocas/vezes que o modelo passará por todo o conjunto de treinamento.
# O número de amostras usado em cada atualização do gradiente.
# Treinamento do Modelo
modelo_p.fit(X_train, y_train, batch_size=32, epochs=100) 

Epoch 1/100
59/59 [==============================] - 1s 3ms/step - loss: 0.7166 - accuracy: 0.5170
Epoch 2/100
59/59 [==============================] - 0s 3ms/step - loss: 0.7093 - accuracy: 0.5034
Epoch 3/100
59/59 [==============================] - 0s 3ms/step - loss: 0.7032 - accuracy: 0.5204
Epoch 4/100
59/59 [==============================] - 0s 3ms/step - loss: 0.6977 - accuracy: 0.5374
Epoch 5/100
59/59 [==============================] - 0s 3ms/step - loss: 0.6932 - accuracy: 0.5306
Epoch 6/100
59/59 [==============================] - 0s 3ms/step - loss: 0.6891 - accuracy: 0.5442
Epoch 7/100
59/59 [==============================] - 0s 3ms/step - loss: 0.6851 - accuracy: 0.5646
Epoch 8/100
59/59 [==============================] - 0s 3ms/step - loss: 0.6816 - accuracy: 0.5408
Epoch 9/100
59/59 [==============================] - 0s 3ms/step - loss: 0.6783 - accuracy: 0.5544
Epoch 10/100
59/59 [==============================] - 0s 3ms/step - loss: 0.6750 - accuracy: 0.5340
Epoch 11/

59/59 [==============================] - 0s 6ms/step - loss: 0.5428 - accuracy: 0.8027
Epoch 84/100
59/59 [==============================] - 0s 7ms/step - loss: 0.5416 - accuracy: 0.8027
Epoch 85/100
59/59 [==============================] - 0s 6ms/step - loss: 0.5406 - accuracy: 0.8027
Epoch 86/100
59/59 [==============================] - 0s 5ms/step - loss: 0.5394 - accuracy: 0.8027
Epoch 87/100
59/59 [==============================] - 0s 3ms/step - loss: 0.5384 - accuracy: 0.8027
Epoch 88/100
59/59 [==============================] - 0s 3ms/step - loss: 0.5375 - accuracy: 0.8027
Epoch 89/100
59/59 [==============================] - 0s 3ms/step - loss: 0.5364 - accuracy: 0.8027
Epoch 90/100
59/59 [==============================] - 0s 3ms/step - loss: 0.5352 - accuracy: 0.8027
Epoch 91/100
59/59 [==============================] - 0s 3ms/step - loss: 0.5342 - accuracy: 0.7993
Epoch 92/100
59/59 [==============================] - 0s 3ms/step - loss: 0.5332 - accuracy: 0.7959
Epoch 93/100


In [25]:
#MLP

#Camadas densas (totalmente conectadas) com 6 e 3 neurônios, respectivamente, nas camadas ocultas. 
modelo_mlp = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Camada de entrada com o número de características
    tf.keras.layers.Dense(6, activation='relu'),  # ReLU é usada nas camadas ocultas para introduzir não-linearidades.
    tf.keras.layers.Dense(3, activation='relu'),  # Camada oculta com ativação ReLU
    tf.keras.layers.Dense(1, activation='sigmoid')  # Camada de saída com 1 neurônio (classificação binária)
])

modelo_mlp.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Treinamento do Modelo
modelo_mlp.fit(X_train, y_train, batch_size=32, epochs=100)


Epoch 1/100
10/10 [==============================] - 2s 3ms/step - loss: 0.7209 - accuracy: 0.4932
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 0.7105 - accuracy: 0.4932
Epoch 3/100
10/10 [==============================] - 0s 4ms/step - loss: 0.7028 - accuracy: 0.4932
Epoch 4/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6967 - accuracy: 0.4932
Epoch 5/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6906 - accuracy: 0.4932
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6854 - accuracy: 0.4966
Epoch 7/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6807 - accuracy: 0.5000
Epoch 8/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6773 - accuracy: 0.5102
Epoch 9/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6735 - accuracy: 0.5272
Epoch 10/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6698 - accuracy: 0.5646
Epoch 11/

10/10 [==============================] - 0s 7ms/step - loss: 0.5184 - accuracy: 0.7959
Epoch 84/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5179 - accuracy: 0.7857
Epoch 85/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5169 - accuracy: 0.8061
Epoch 86/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5141 - accuracy: 0.8197
Epoch 87/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5128 - accuracy: 0.8163
Epoch 88/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5105 - accuracy: 0.8163
Epoch 89/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5100 - accuracy: 0.8095
Epoch 90/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5079 - accuracy: 0.8163
Epoch 91/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5062 - accuracy: 0.8197
Epoch 92/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5044 - accuracy: 0.8163
Epoch 93/100


In [18]:
# Modelo de Árvore de Decisão
modelo_dt = DecisionTreeClassifier()
modelo_dt.fit(X_train, y_train)

# Previsões no conjunto de teste
y_pred_dt = modelo_dt.predict(X_test)

In [19]:
# Perceptron Simples
y_pred_p = modelo_p.predict(X_test)
y_pred_p = np.where(y_pred_p >= 0.5, 1, 0)
df_comparacao_p = pd.DataFrame(y_test)
df_comparacao_p["Predição"] = y_pred_p
print("Comparação - Perceptron Simples")
print(df_comparacao_p)
accuracy_p = accuracy_score(y_test, y_pred_p)
precision_p = precision_score(y_test, y_pred_p)
recall_p = recall_score(y_test, y_pred_p)
f1_p = f1_score(y_test, y_pred_p)

##################################################################################################################

# MLP
y_pred_mlp = modelo_mlp.predict(X_test)
y_pred_mlp = np.where(y_pred_mlp >= 0.5, 1, 0)
df_comparacao_mlp = pd.DataFrame(y_test)
df_comparacao_mlp["Predição"] = y_pred_mlp
print("Comparação - Perceptron Multi Camadas")
print(df_comparacao_mlp)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
precision_mlp = precision_score(y_test, y_pred_mlp)
recall_mlp = recall_score(y_test, y_pred_mlp)
f1_mlp = f1_score(y_test, y_pred_mlp)

#######################################################################################################

# Árvore de Decisão
y_pred_dt = modelo_dt.predict(X_test)
df_comparacao_dt = pd.DataFrame(y_test)
df_comparacao_dt["Predição"] = y_pred_dt
print("Comparação - Árvore de Decisão")
print(df_comparacao_dt)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt)
recall_dt = recall_score(y_test, y_pred_dt)
f1_dt = f1_score(y_test, y_pred_dt)

# Impressão das métricas
print(f"Perceptron Simples\nAcurácia: {accuracy_p}\nPrecisão: {precision_p}\nRecall: {recall_p}\nF1-score: {f1_p}\n")
print(f"Perceptron Multicamadas\nAcurácia: {accuracy_mlp}\nPrecisão: {precision_mlp}\nRecall: {recall_mlp}\nF1-score: {f1_mlp}\n")
print(f"Árvore de Decisão\nAcurácia: {accuracy_dt}\nPrecisão: {precision_dt}\nRecall: {recall_dt}\nF1-score: {f1_dt}\n")

4/4 [==============================] - 0s 4ms/step
Comparação - Perceptron Simples
     class  Predição
220      0         1
297      0         1
307      0         1
290      0         1
35       1         0
..     ...       ...
258      0         1
276      0         1
285      0         1
236      0         1
59       1         1

[126 rows x 2 columns]
4/4 [==============================] - 0s 4ms/step
Comparação - Perceptron Multi Camadas
     class  Predição
220      0         0
297      0         0
307      0         0
290      0         0
35       1         0
..     ...       ...
258      0         0
276      0         0
285      0         0
236      0         0
59       1         0

[126 rows x 2 columns]
Comparação - Árvore de Decisão
     class  Predição
220      0         0
297      0         0
307      0         0
290      0         0
35       1         1
..     ...       ...
258      0         0
276      0         0
285      0         0
236      0         0
59       1    